# Vessel Route Optimization

In [1]:
import math
import numpy as np
from numpy import random
from scipy.optimize import fsolve
import ortools
from tabulate import tabulate
from ortools.linear_solver import pywraplp

## Semi-random vectors generation

In [2]:
wind_degrees = []
x_positions = 10
max_change = 0.3
#random.seed(1234)

for day in range(x_positions):
    if day == 0:
        wind_degrees.append(random.randint(0,360))
    else:
        last_day = wind_degrees[day-1]
        wind_degrees.append(random.randint(last_day*(1-max_change),last_day*(1+max_change)))
        
wind_speed = list(random.randint(0,28,(1,x_positions)).flatten())

print("Wind direction (degrees):", wind_degrees)
print("Wind speed (m/s):", wind_speed)

Wind direction (degrees): [59, 64, 74, 81, 72, 71, 56, 55, 51, 50]
Wind speed (m/s): [17, 8, 26, 1, 16, 2, 21, 15, 13, 27]


## Multiple array problem

In [16]:
def random_space(max_value, x_positions=5, y_positions=1, day_max_change=0.3, position_max_change=0.2, seed=None):    
    random.seed(seed)
    degrees_matrix = []
    wind_degrees = []
    
    for day in range(x_positions):
        wind_degrees.append(random.randint(1,max_value))
    degrees_matrix.append(wind_degrees) 

    for position in range(0,y_positions-1):
        position_degree = []
        for day in range(x_positions):
            same_day_other_position = abs(degrees_matrix[position-1][day])  
            if same_day_other_position != 0:
                position_degree.append(random.randint(same_day_other_position*(1-position_max_change),same_day_other_position*(1+position_max_change)))
            else: 
                position_degree.append(random.randint(1,max_value))
        degrees_matrix.append(position_degree)
    
    return degrees_matrix

x_positions = 5
y_positions = 3
day_max_change = 0.3
position_max_change = 0.3
seed = 126

wind_direction = random_space(360, x_positions, y_positions, day_max_change, position_max_change)
wind_speed = random_space(27, x_positions, y_positions, day_max_change, position_max_change)

print("Wind direction (degrees):", wind_direction)
print("Wind speed (m/s):", wind_speed)

Wind direction (degrees): [[115, 62, 91, 13, 305], [141, 52, 69, 10, 293], [120, 66, 99, 10, 386]]
Wind speed (m/s): [[1, 8, 16, 2, 16], [0, 6, 15, 1, 14], [0, 7, 14, 1, 18]]


# Open_sail function

In [4]:
#CONSTANTS TO BE DETERMINED
min_wind_angle = 60 
max_wind_angle = 300 
min_wind_strength = 10

def open_sail(wind_angle, wind_strength):
    open_sail_bool = False 
    
    if wind_angle > min_wind_angle and wind_angle < max_wind_angle and wind_strength > min_wind_strength :
        open_sail_bool = True
        
    #EXCLUDING CRIETRIA FOR THE BOAT_SPEED FUNCTION NOT TO EXPLODE
    if wind_angle > 80 and wind_angle < 120 : open_sail_bool = False
    if wind_angle > 260 and wind_angle < 300 : open_sail_bool = False


    return open_sail_bool


print("We open the sail : ", open_sail(80,40))

We open the sail :  True


# Apparent wind function for boat speed

In [5]:
# 0<a0<180

def boat_speed(a0,VW,eta): 

    f = lambda a: math.sin(math.pi*a0/180) * math.sin(math.pi*a/180) * math.pow((math.sin(math.pi*a/180/2) / math.sin(math.pi*a0/180-math.pi*a/180)), 2)-VW * eta
    a = fsolve(f, [0])[0]
    #print("a:", a)
    VB = VW * (math.sin(math.pi*(a0-a)/180)/math.sin(math.pi*a/180))
    #print("a0:",a0,"and VB:", VB)
    
    return VB

# Problem in 1 dimension

In [6]:
#ARBITRARY CONSTANTS TO CHECK !!!
total_distance = 30 #miles
x_positions = 6 #nb of squares between A and B
eta = 0.05
motor_speed = 20 #in mph
motor_emission_per_mile = 3.2 #in kgCO2 per mile

#generating random field
wind_angle = random_space(360,1,x_positions) #in degrees
wind_speed = random_space(30,1,x_positions) #in mph
square_length = total_distance/x_positions

#defining the lists to be used
list_co2 = []
list_time = []
list_sail_speed = []
list_motor_percentage = []
list_boat_speed = []

#flatten the list of list into a simple list
wind_angle = [item for sublist in wind_angle for item in sublist]
wind_speed = [item for sublist in wind_speed for item in sublist]

#evaluating the increase of speed we can reach thanks to sails 
for i in range(x_positions):
    if open_sail(wind_angle[i],wind_speed[i]):
        speed = boat_speed(wind_angle[i],wind_speed[i],eta)
        if 10 > speed > 3 :
            list_sail_speed.append(speed)
        else: 
            list_sail_speed.append(0)
    else: list_sail_speed.append(0)

#calculating the power needed of the motor in % to reach 20mph
list_motor_percentage = [((motor_speed-i)/motor_speed) for i in list_sail_speed]

#calculating the sum of the speeds (sail + motor)
list_boat_speed = [list_sail_speed[i] + (list_motor_percentage[i]*motor_speed) for i in range(len(list_sail_speed))]
 
#CO2 emissions calculations
for i in range(x_positions):
    list_co2.append(motor_emission_per_mile*list_motor_percentage[i]*square_length)
    
total_emissions = sum(list_co2)
        
#determining the time needed for the journey
for i in range(x_positions):
    list_time.append(square_length/list_boat_speed[i])
        
total_time = sum(list_time)        
      
    
#printing results
print("The squares length is", square_length)
print("Wind angles are :", wind_angle)
print("Wind speeds are :", wind_speed)

print("Speed gained from sails:", list_sail_speed)
print("Motor usage:",  list_motor_percentage)
print("Boat speeds are :", list_boat_speed)

print("The emissions for each square are:", list_co2)
print("The total emissions are:", total_emissions)
print("The time need for each square is:", list_time)
print("The total time needed is:", total_time)

The squares length is 5.0
Wind angles are : [108, 116, 118, 118, 119, 140]
Wind speeds are : [7, 5, 6, 5, 5, 4]
Speed gained from sails: [0, 0, 0, 0, 0, 0]
Motor usage: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Boat speeds are : [20.0, 20.0, 20.0, 20.0, 20.0, 20.0]
The emissions for each square are: [16.0, 16.0, 16.0, 16.0, 16.0, 16.0]
The total emissions are: 96.0
The time need for each square is: [0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
The total time needed is: 1.5


# Problem in 2 dimension

In [17]:
#ARBITRARY CONSTANTS TO CHECK !!!
total_distance = 60 #miles
x_positions = 10 #nb of squares between A and B
y_positions = 10
eta = 0.05
motor_speed = 15 #in mph
motor_emission_per_mile = 3.2 #in kgCO2 per mile

wind_angle_matrix = random_space(360, x_positions, y_positions) #in degrees
wind_speed_matrix = random_space(30, x_positions, y_positions) #in mph


def single_line(wind_angle, wind_speed, print_=False):
    
    #defining the lists to be used
    list_co2 = []
    list_time = []
    list_sail_speed = []
    list_motor_percentage = []
    list_boat_speed = []

    #evaluating the increase of speed we can reach thanks to sails 
    for i in range(x_positions):
        if open_sail(wind_angle[i],wind_speed[i]):
            speed = boat_speed(wind_angle[i],wind_speed[i],eta)
            if 10 > speed > 3 :
                list_sail_speed.append(speed)
            else: 
                list_sail_speed.append(0)
        else: list_sail_speed.append(0)

    #calculating the power needed of the motor in % to reach 20mph
    list_motor_percentage = [((motor_speed-i)/motor_speed) for i in list_sail_speed]

    #calculating the sum of the speeds (sail + motor)
    list_boat_speed = [list_sail_speed[i] + (list_motor_percentage[i]*motor_speed) for i in range(len(list_sail_speed))]

    #CO2 emissions calculations
    for i in range(x_positions):
        list_co2.append(motor_emission_per_mile*list_motor_percentage[i]*square_length)

    total_emissions = sum(list_co2)

    #determining the time needed for the journey
    for i in range(x_positions):
        list_time.append(square_length/list_boat_speed[i])

    total_time = sum(list_time)
    
    #printing results
    if print_:
        print("The squares length is", square_length)
        print("Wind angles are :", wind_angle)
        print("Wind speeds are :", wind_speed)

        print("Speed gained from sails:", list_sail_speed)
        print("Motor usage:",  list_motor_percentage)
        print("Boat speeds are :", list_boat_speed)

        print("The emissions for each square are:", list_co2)
        print("The total emissions are:", total_emissions)
        print("The time need for each square is:", list_time)
        print("The total time needed is:", total_time)
        
    return (wind_angle, wind_speed, list_sail_speed, list_boat_speed, list_co2, list_motor_percentage)

path = np.zeros((y_positions, x_positions), int)
result_wind_angle = []
result_wind_speed = []
result_sail_speed = []
result_boat_speed = []
result_co2 = []
result_motor_percentage = []

for y in range(y_positions):
    results = single_line(wind_angle_matrix[y], wind_speed_matrix[y])
    result_wind_angle.append(results[0])
    result_wind_speed.append(results[1])
    result_sail_speed.append(results[2])
    result_boat_speed.append(results[3])
    result_co2.append(results[4])
    result_motor_percentage.append(results[5])
    
result_co2

[[5.994087538411876,
  16.0,
  16.0,
  16.0,
  16.0,
  16.0,
  16.0,
  6.378042037905476,
  16.0,
  16.0],
 [7.484472223398866,
  16.0,
  16.0,
  16.0,
  16.0,
  16.0,
  16.0,
  5.895296129415533,
  16.0,
  16.0],
 [6.480879677766423,
  16.0,
  16.0,
  16.0,
  16.0,
  16.0,
  16.0,
  6.921215557322817,
  5.440370430894367,
  16.0],
 [16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0],
 [7.006805251541456, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0],
 [16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0],
 [7.2463809033665685,
  16.0,
  16.0,
  16.0,
  6.403989126690428,
  16.0,
  16.0,
  16.0,
  16.0,
  16.0],
 [16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0],
 [16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0],
 [16.0, 16.0, 16.0, 16.0, 5.474477042891389, 16.0, 16.0, 16.0, 16.0, 16.0]]

# Solver Model

In [18]:
solver = pywraplp.Solver.CreateSolver('CLP')

x = {}
for i in range(y_positions):
    for j in range(x_positions):
        x[i, j] = solver.IntVar(0, 1, '')
        
# Constraints

middle_point = y_positions // 2  

# for each x_position, we need the boat to pass by there
for j in range(x_positions):
    solver.Add(solver.Sum([x[i, j] for i in range(y_positions)]) == 1)

# you can only move either the square in front of you either in the two closest diagonals        
r = [middle_point-1, middle_point, middle_point+1]    
    
for j in range(1, x_positions):
    for i in range(y_positions-1):
        if i > 0 & i < y_positions-1:
            solver.Add(x[i,j] <= x[i,j-1] + x[i-1,j-1] + x[i+1,j-1])
        if i == 0:
            solver.Add(x[i,j] <= x[i,j-1] + x[i+1,j-1])
        if i == y_positions-1:
            solver.Add(x[i,j] <= x[i,j-1] + x[i-1,j-1])
        if j == 1:
            solver.Add(solver.Sum([x[i, j] for i in r]) == 1) 

  
#choose a specific starting point (i)
solver.Add(x[middle_point,0] == 1)
#chose a specific ending point (i)
solver.Add(x[middle_point,x_positions-1] == 1)

def get_distance(starting_point, end_point):
    return math.sqrt((end_point[0]-starting_point[0])**2 + (end_point[1]-starting_point[1])**2)

solver.Minimize(solver.Sum([result_co2[i][j] * x[i,j] for i in range(y_positions)
                                                  for j in range(1,x_positions)]))
status = solver.Solve()

In [9]:
status

NameError: name 'status' is not defined

In [81]:
# Print solution.
if status == pywraplp.Solver.OPTIMAL:
    print('Total co2 = ', solver.Objective().Value(), '\n')
    for j in range(x_positions):
        for i in range(y_positions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print('x_position %d took y_position %d.  CO2 = %d' %
                  (j, i, result_co2[i][j]))

Total co2 =  110.14684590321852 

x_position 0 took y_position 5.  CO2 = 16
x_position 1 took y_position 4.  CO2 = 16
x_position 2 took y_position 5.  CO2 = 16
x_position 3 took y_position 6.  CO2 = 6
x_position 4 took y_position 5.  CO2 = 6
x_position 5 took y_position 4.  CO2 = 8
x_position 6 took y_position 3.  CO2 = 8
x_position 7 took y_position 3.  CO2 = 16
x_position 8 took y_position 4.  CO2 = 16
x_position 9 took y_position 5.  CO2 = 16


In [82]:
#create a variavle path that store the solution from the solver
for j in range(x_positions):
        for i in range(y_positions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            path[i,j] = (x[i,j].solution_value())

# function to represent the solution graphically            
def show_table(path):
    results = path.tolist()          
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i][j] == 1:
                results[i][j] = "#"
            elif results[i][j] == 0:
                results[i][j] = ""
    print(tabulate(list(map(tuple, results)), tablefmt="grid", stralign="center"))

show_table(path)

+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   | # | # |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   | # |   |   |   | # |   |   | # |   |
+---+---+---+---+---+---+---+---+---+---+
| # |   | # |   | # |   |   |   |   | # |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   | # |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+


# Creating measures of the field to calculate distance and velocity

In [83]:
# define hight of the map
tot_y_distance = 60 #miles
tot_x_distance = total_distance #miles
y_square = tot_y_distance/y_positions
x_square = tot_x_distance/x_positions

In [84]:
coordinates_field = [[(y_square*i,x_square*j) for j in range(x_positions)] for i in range(y_positions)]
print(tabulate(coordinates_field, tablefmt="grid", stralign="center"))

+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+
| (0.0, 0.0)  | (0.0, 6.0)  | (0.0, 12.0)  | (0.0, 18.0)  | (0.0, 24.0)  | (0.0, 30.0)  | (0.0, 36.0)  | (0.0, 42.0)  | (0.0, 48.0)  | (0.0, 54.0)  |
+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+
| (6.0, 0.0)  | (6.0, 6.0)  | (6.0, 12.0)  | (6.0, 18.0)  | (6.0, 24.0)  | (6.0, 30.0)  | (6.0, 36.0)  | (6.0, 42.0)  | (6.0, 48.0)  | (6.0, 54.0)  |
+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+
| (12.0, 0.0) | (12.0, 6.0) | (12.0, 12.0) | (12.0, 18.0) | (12.0, 24.0) | (12.0, 30.0) | (12.0, 36.0) | (12.0, 42.0) | (12.0, 48.0) | (12.0, 54.0) |
+-------------+-------------+--------------+--------------+--------------+--------------+-----------

In [85]:
# we start from (10,0)
start_point = coordinates_field[1][0]
# we go straight
straight_point = coordinates_field[1][1]
# we go diagonally
diagonal_point = coordinates_field[0][1]

def get_distance(starting_point, end_point):
    return math.sqrt((end_point[0]-starting_point[0])**2 + (end_point[1]-starting_point[1])**2)

print("going straight:", get_distance(start_point, straight_point))
print("going diagonally:", get_distance(start_point, diagonal_point))

going straight: 6.0
going diagonally: 8.48528137423857


In [86]:
def get_path_distance(path, coordinates_field):
    passed_by = []
    for j in range(x_positions):
        for i in range(y_positions):
            if path[i][j] == 1:
                passed_by.append(coordinates_field[i][j])
    total_distances = [0]            
    for way in range(len(passed_by)-1):
        total_distances.append(get_distance(passed_by[way], passed_by[way+1]))
    return total_distances

print("total distance: {} miles".format(sum(get_path_distance(path, coordinates_field))))

total distance: 73.88225099390856 miles


# Results

In [ ]:
#calculate the resulting emission
co2_path = path*result_co2
co2_consumption = co2_path[co2_path!=0]
co2_consumption

In [ ]:
passed_by = np.zeros((y_positions, x_positions), int)
passed_by[i,j] = path[i,j] + passed_by[i,j]
get_distance(passed_by[1], passed_by[1+1])

In [ ]:
def get_distance(starting_point, end_point):
    return math.sqrt((end_point[0]-starting_point[0])**2 + (end_point[1]-starting_point[1])**2)

In [19]:
from ortools.linear_solver import pywraplp

# define the solver
solver = pywraplp.Solver('SolveShortestPathProblem',
                         pywraplp.Solver.CLP_LINEAR_PROGRAMMING)

# define the matrix
matrix = [[5, 2, 3, 4, 5, 6],
          [5, 2, 3, 4, 7, 6],
          [5, 4, 5, 0, 5, 6],
          [6, 6, 5, 4, 1, 10],
          [8, 2, 3, 6, 4, 6],
          [9, 2, 3, 4, 6, 6]]

# define the variables
path = {}
for i in range(6):
    for j in range(6):
        path[i, j] = solver.IntVar(0, 1, 'path[%i,%i]' % (i, j))

# define the constraints
# each position can be visited only once
for i in range(6):
    for j in range(6):
        solver.Add(solver.Sum([path[i, j] for i in range(6) for j in range(6)]) == 1)

# the path must start at position (3,0) and end at position (3,5)
solver.Add(path[3, 0] == 1)
solver.Add(path[3, 5] == 1)

# the path must pass through each column exactly once
for j in range(6):
    solver.Add(solver.Sum([path[i, j] for i in range(6)]) == 1)

# the path can only move in straight or diagonal directions
for i in range(6):
    for j in range(6):
        if i > 0:
            solver.Add(path[i, j] <= path[i - 1, j])
        if i < 5:
            solver.Add(path[i, j] <= path[i + 1, j])
        if j > 0:
            solver.Add(path[i, j] <= path[i, j - 1])
        if j < 5:
            solver.Add(path[i, j] <= path[i, j + 1])
        if i > 0 and j > 0:
            solver.Add(path[i, j] <= path[i - 1, j - 1])
        if i > 0 and j < 5:
            solver.Add(path[i, j] <= path[i - 1, j + 1])
        if i < 5 and j > 0:
            solver.Add(path[i, j] <= path[i + 1, j - 1])
        if i < 5 and j < 5:
            solver.Add(path[i, j] <= path[i + 1, j + 1])


In [20]:
# minimize the total distance of the path
total_distance = solver.Sum([matrix[i][j] * path[i, j]
                             for i in range(6)
                             for j in range(6)])
solver.Minimize(total_distance)

# solve the problem
status = solver.Solve()

# print the results
if status == pywraplp.Solver.OPTIMAL:
    print('Minimum distance:', solver.Objective().Value())
    for i in range(6):
        for j in range(6):
            if path[i, j].solution_value() > 0:
                print('Position (%i, %i) is part of the path.' % (i, j))
else:
    print('The problem does not have an optimal solution.')

The problem does not have an optimal solution.


In [56]:
#ARBITRARY CONSTANTS TO CHECK !!!
total_distance = 60 #miles
x_positions = 7 #nb of squares between A and B
y_positions = 5
eta = 0.05
motor_speed = 20 #in mph
motor_emission_per_mile = 3.2 #in kgCO2 per mile

wind_angle_matrix = random_space(360, x_positions, y_positions) #in degrees
wind_speed_matrix = random_space(30, x_positions, y_positions) #in mph


def single_line(wind_angle, wind_speed, print_=False):
    
    #defining the lists to be used
    list_sail = []
    list_boat_speed = []
    list_co2 = []
    list_time = []
    
    for i in range(x_positions):
        #evaluating if we open the sails for each position
        list_sail.append(open_sail(wind_angle[i],wind_speed[i]))

        #determining the boat speeds depending on whether we use sails on motor   
        if list_sail[i]==True:
            list_boat_speed.append(boat_speed(wind_angle[i],wind_speed[i],eta))

        else:
            list_boat_speed.append(motor_speed)

        #CO2 emissions calculations    
        if list_sail[i]==True:
            list_co2.append(0)

        else:
            list_co2.append(motor_emission_per_mile*square_length)

        #determining the time needed for the journey    
        list_time.append(square_length/list_boat_speed[i])    

    #printing results
    if print_:
        print("Wind angles are :", wind_angle)
        print("Wind speeds are :", wind_speed)
        print("We open the sails :", list_sail)
        print("Boat speeds are :", list_boat_speed)
    return (wind_angle,wind_speed, list_sail,list_boat_speed, list_co2)

path = np.zeros((y_positions, x_positions), int)
result_wind_angle = []
result_wind_speed = []
result_sail = []
result_boat_speed = []
result_co2 = []

for y in range(y_positions):
    results = single_line(wind_angle_matrix[y], wind_speed_matrix[y])
    result_wind_angle.append(results[0])
    result_wind_speed.append(results[1])
    result_sail.append(results[2])
    result_boat_speed.append(results[3])
    result_co2.append(results[4])
    
result_co2


result_co2[1][2] =16.
result_co2


## New test

solver = pywraplp.Solver.CreateSolver('CLP')

x = {}
for i in range(y_positions):
    for j in range(x_positions):
        x[i, j] = solver.IntVar(0, 1, '')

# Define the constraints
middle_point = middle_point = y_positions // 2 
# The starting and ending points are fixed
solver.Add(x[middle_point,0] == 1)
solver.Add(x[middle_point,x_positions-1] == 1)


r = [middle_point-1, middle_point, middle_point+1]
print(r)


# For each column, only one variable can take the value 1
for j in range(x_positions):
    solver.Add(solver.Sum([x[i,j] for i in range(y_positions)]) == 1)
    # Force second column constraint to follow from middle point and be have only allowed potential solutions
    


# For each column, the only allowed movements from the previous column are
# to the same row, the up-diagonal, or the low-diagonal
for j in range(1, x_positions):
    for i in range(y_positions):
        if i > 0 and i < y_positions-1:
            solver.Add(x[i,j] <= x[i,j-1] + x[i-1,j-1] + x[i+1,j-1])
        if i == 0:
            solver.Add(x[i,j] <= x[i,j-1] + x[i+1,j-1])
        if i == y_positions-1:
            solver.Add(x[i,j] <= x[i,j-1] + x[i-1,j-1])
        if j == 1:
            solver.Add(solver.Sum([x[i, j] for i in r]) == 1)


# for each x_position, we need the boat to pass by there
for j in range(x_positions):
    solver.Add(solver.Sum([x[i, j] for i in range(y_positions)]) == 1)





solver.Minimize(solver.Sum([result_co2[i][j] * x[i,j] for i in range(y_positions)
                                                  for j in range(x_positions)]))
status = solver.Solve()



# Print solution.
if status == pywraplp.Solver.OPTIMAL:
    print('Total co2 = ', solver.Objective().Value(), '\n')
    for j in range(x_positions):
        for i in range(y_positions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() >= 0.5:
                print('x_position %d took y_position %d.  CO2 = %d' %
                      (j, i, result_co2[i][j]))


for j in range(x_positions):
        for i in range(y_positions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            path[i,j] = (x[i,j].solution_value())

def show_table(path):
    results = path.tolist()          
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i][j] == 1:
                results[i][j] = "#"
            elif results[i][j] == 0:
                results[i][j] = ""
    print(tabulate(list(map(tuple, results)), tablefmt="grid", stralign="center"))

show_table(path)

[1, 2, 3]
Total co2 =  80.0 

x_position 0 took y_position 2.  CO2 = 16
x_position 1 took y_position 1.  CO2 = 16
x_position 2 took y_position 0.  CO2 = 0
x_position 2 took y_position 2.  CO2 = 0
x_position 3 took y_position 2.  CO2 = 16
x_position 3 took y_position 3.  CO2 = 16
x_position 4 took y_position 3.  CO2 = 0
x_position 5 took y_position 2.  CO2 = 16
x_position 6 took y_position 2.  CO2 = 16
+---+---+--+--+---+---+---+
|   |   |  |  |   |   |   |
+---+---+--+--+---+---+---+
|   | # |  |  |   |   |   |
+---+---+--+--+---+---+---+
| # |   |  |  |   | # | # |
+---+---+--+--+---+---+---+
|   |   |  |  | # |   |   |
+---+---+--+--+---+---+---+
|   |   |  |  |   |   |   |
+---+---+--+--+---+---+---+


In [41]:
result_co2

[[16.0, 0, 16.0, 0, 16.0, 0, 0],
 [16.0, 0, 16.0, 0, 16.0, 0, 0],
 [16.0, 0, 16.0, 0, 16.0, 0, 16.0],
 [16.0, 16.0, 16.0, 0, 16.0, 0, 16.0],
 [16.0, 16.0, 16.0, 0, 16.0, 0, 0]]

In [60]:
for j in range(x_positions):
    print("\n")
    for i in range(y_positions):
               print(x[i,j].solution_value())
                
                



0.0
0.0
1.0
0.0
0.0


0.0
1.0
0.0
0.0
0.0


0.5
0.0
0.5
0.0
0.0


0.0
0.0
0.5
0.5
0.0


0.0
0.0
0.0
1.0
0.0


0.0
0.0
1.0
0.0
0.0


0.0
0.0
1.0
0.0
0.0


In [53]:
path

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])